This file is used for generating Spectral Power Vectors (SPV) from *Raw* features. The *Raw* EEG features are obtained using `matlab_files/eeg_csp_raw_run.m`. Refer Section III C(1) in the paper for feature extraction method. The highlighted part in the following image indicate the pipeline of the current file.

Note: The configurations, like hyperparameter values, present in this file are not the final configurations as reported in the paper. The present configurations are maybe mutated for additional experiments. However, results reported in the paper can be reproduced by plugging in the appropriate configurations as mentioned in the paper. In this code `NC` means Normal Controls (equivalent to `HC` as referred in the paper)

![Generating Spectral Power Vectors from EEG Signals](images/raw_SPV.png)

(Optional) Mount Gdrive on Google Colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Imports

In [ ]:
from scipy.signal import butter
from scipy.signal import filtfilt
from scipy.signal import iirfilter, lfilter
from scipy.fft import fft
import os
import xlrd
import pandas as pd
import numpy as np
from scipy.io import loadmat, savemat
from math import floor
from tqdm.notebook import tqdm
from sklearn.decomposition import PCA
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, accuracy_score, make_scorer
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV 
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
import os
import pandas as pd
from math import floor
import numpy as np
from scipy.stats import sem
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from matplotlib import pyplot

# Signal Processing

In [ ]:
def amplitude_reduction(data, threshold):
  data[data >= threshold] = 0
  data[data <= -threshold] = 0
  #normalized = (data-min(data))/(max(data)-min(data)) # normalize
  return data

# butter band_pass_filter
def butter_bandpass_filter(data, lowcut, highcut, fs, order=7):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    #b, a = iirfilter(N=order, Wn = [low, high], btype='bandpass', ftype='butter', fs=fs)
    out = lfilter(b, a, data)
    #data = amplitude_reduction(data, 85) # amplitude thresholding
    #out = filtfilt(b, a, data)
    return out

# alpha spectral band
def alpha_BP(data):
    return butter_bandpass_filter(data, 8, 13, 128)

# Beta spectral band
def beta_BP(data):
    return butter_bandpass_filter(data, 13, 30, 128)

# Gamma spectral band
def gamma_BP(data):
    return butter_bandpass_filter(data, 30, 49, 128)


# Band power AKA sum of squares of Fast Fourier Transform (FFT) coefficients

def average_intensity(band):
    return sum(np.square(abs(band)))


# Alpha Beta Gamma Calculation
def alpha_beta_gamma_calc(data_eeg,max_channel):
    alphas = []
    betas = []
    gammas = []

    for i in range(0,max_channel,1):
        data = data_eeg[i,:]
        alphas.append(average_intensity(fft(alpha_BP(data))))
        betas.append(average_intensity(fft(beta_BP(data))))
        gammas.append(average_intensity(fft(gamma_BP(data))))

    return np.concatenate((alphas,betas, gammas), axis=None)


# Feature Extraction Function
def feature_eeg_xl(dir, seconds, sampling_rate):
    final_feature_mat = []
    y_pd_nc = []
    y_emotions = []
    for root, dire, filename in tqdm(sorted(os.walk(dir))):
      for files in (filename):
        if files.endswith(".xls") or files.endswith(".xlsx"): 
            #print('Genrating Features for file = ',root, dire, files)
            M_data = pd.read_excel(os.path.join(root, files), header=None)
            eeg_data = np.transpose(M_data.to_numpy())
            epochs = floor(eeg_data.shape[1]/(seconds*sampling_rate))
            for i in range(epochs):
              eeg_split_data = eeg_data[:, (i)*seconds*sampling_rate:(i+1)*seconds*sampling_rate]
              final_feature_mat.append(alpha_beta_gamma_calc(eeg_split_data, 14))
        else:
            continue
    return np.array(final_feature_mat)

# Feature Extraction

In [ ]:
dir1 = "/content/drive/MyDrive/PD_NC DATABASE_FINAL"
PD_NC = ['NC', 'PD']
Emotions = ['E1','E2','E3','E4','E5','E6']
#Emotions = ['E1']
seconds = 5
sampling_rate =128
feature_mat = np.empty((0,42))
y_vec_emot = np.array([])
y_vec_pdnc = np.array([])

for pdnc in PD_NC:
  for emot in Emotions:
    dir2 = os.path.join(dir1, pdnc, emot)
    emot_mat = feature_eeg_xl(dir2, seconds, sampling_rate)
    y_vec_emot = np.concatenate((y_vec_emot, [emot]*emot_mat.shape[0]), axis = None)
    y_vec_pdnc = np.concatenate((y_vec_pdnc, [pdnc]*emot_mat.shape[0]), axis = None)
    feature_mat = np.concatenate((feature_mat, emot_mat), axis = 0)

In [ ]:
print(feature_mat.shape)
print(y_vec_emot.shape)
print(y_vec_pdnc.shape)

(14800, 42)
(14800,)
(14800,)


In [ ]:
full_feat = feature_mat

## Full PD-NC Labels

In [ ]:
# Full PDNC labels
df_y_pdnc = pd.DataFrame(y_vec_pdnc, dtype='string')
labels_pdnc = {'PD':1, 'NC':0}
df_y_pdnc = df_y_pdnc.replace(labels_pdnc)
full_pdnc_labels = df_y_pdnc[0].to_numpy()
full_pdnc_labels

array([0, 0, 0, ..., 1, 1, 1])

## Full Multiclass labels

In [ ]:
# full multi labels
df_y_multi = pd.DataFrame(y_vec_emot, dtype='string')
labels_multi = {'E1': 1, 'E2':2, 'E3':3, 'E4':4, 'E5':5, 'E6':6}
df_y_multi = df_y_multi.replace(labels_multi)
full_multi_labels = df_y_multi[0].to_numpy()
full_multi_labels

array([1, 1, 1, ..., 6, 6, 6])

## Full HV-LV Labels

In [ ]:
#full hvlv labels
df_y_hvlv = pd.DataFrame(y_vec_emot, dtype='string')
labels_hvlv = {'E1':0, 'E2':1, 'E3':0, 'E4':0, 'E5':1, 'E6':0}
df_y_hvlv = df_y_hvlv.replace(labels_hvlv)
full_hvlv_labels = df_y_hvlv[0].to_numpy()
full_hvlv_labels

array([0, 0, 0, ..., 0, 0, 0])

## Fl HA-LA Labels

In [ ]:
# full hala labels

df_y_hala = pd.DataFrame(y_vec_emot, dtype='string')
labels_hala = {'E1':0, 'E2':1, 'E3':1, 'E4':1, 'E5':1, 'E6':1}
df_y_hala = df_y_hala.replace(labels_hala)
full_hala_labels = df_y_hala[0].to_numpy()
full_hala_labels

array([0, 0, 0, ..., 1, 1, 1])

## Sad data and Labels

In [ ]:
# E1 data and labels

df_E = pd.DataFrame(feature_mat)
df_E[len(df_E.columns)] = df_y_pdnc
df_E[len(df_E.columns)] = df_y_multi

df_E1 = df_E.loc[df_E[43] == 1]

E1_feat = df_E1.iloc[:, :42].to_numpy()
E1_pdnc_labels = df_E1.iloc[:, 42].to_numpy()
print(E1_feat)
print(E1_pdnc_labels)

[[2383255.69459581 2384181.35395744 1480504.56993436 ... 2949219.20836618
  5552445.38474222 4276478.68309461]
 [1923588.15676131 1524710.54855611 1440901.95742668 ... 2768476.56285555
  4620527.94896684 4353007.10406718]
 [1668043.90950015 1478329.23855934 1757221.85387915 ... 2248584.72881197
  4169794.55200322 3300814.10692213]
 ...
 [2749607.91064562 5844646.30577007 4003145.13212548 ... 1565742.68789487
  3256600.7290617  2420036.17671516]
 [2875197.80164005 4949090.45825599 4644985.6778534  ... 1610974.59857755
  2821172.40766566 1898007.82120636]
 [3393774.95038035 6377382.46772805 5162434.5054222  ... 1594092.98957601
  3689323.94390404 2177754.47267242]]
[0 0 0 ... 1 1 1]


## Happy data labels

In [ ]:
# E2 data and labels
df_E2 = df_E.loc[df_E[43] == 2]

E2_feat = df_E2.iloc[:, :42].to_numpy()
E2_pdnc_labels = df_E2.iloc[:, 42].to_numpy()
print(E2_feat)
print(E2_pdnc_labels)

[[1550720.59322824 1085128.52405939 1033167.06314666 ... 3016734.88288784
  4332183.93075982 3106802.83523987]
 [1828942.26947362 1356636.0692931  1652012.37909848 ... 2265381.04932823
  4233539.19579308 3535890.12045566]
 [2022056.18662822 1450365.36564622 1413655.63419677 ... 3199325.23870063
  5611345.2212154  4528565.62215991]
 ...
 [3546361.71978832 7276127.55764014 3091380.41164017 ... 2785427.12194215
  4097002.83380655 4686100.01192414]
 [1947817.30150293 3896299.34667278 2865213.2802039  ... 2016508.2428905
  3315486.8643367  3642759.85133093]
 [3251616.78972959 4946439.17129576 3771393.32494778 ... 2334554.86379254
  3911173.74390322 4225805.86176526]]
[0 0 0 ... 1 1 1]


## Fear data and labels

In [ ]:
# E3 data and labels
df_E3 = df_E.loc[df_E[43] == 3]

E3_feat = df_E3.iloc[:, :42].to_numpy()
E3_pdnc_labels = df_E3.iloc[:, 42].to_numpy()
print(E3_feat)
print(E3_pdnc_labels)

[[17078555.89986517 17327468.91727593 18564300.60631271 ...
   4348870.33459041  5250578.01738298  4153439.12593631]
 [ 4967299.31999924  4826399.78559697  4065201.91784005 ...
   2673272.45331669  3944355.15017598  3854599.53346922]
 [ 2997392.10896896  4210987.71627565  2196495.79308463 ...
   2782409.11985911  4735440.81723979  3836436.29534367]
 ...
 [ 1843842.50568231  5959086.42831018  4277417.13896814 ...
   2433384.15481127  4446757.74011414  2818030.8089374 ]
 [ 1592850.53851936  6831639.90141186  4016541.40470296 ...
   2145017.49129135  2953252.22748077  2128168.20846729]
 [ 1628536.90593345  3949214.12519319  3995445.68923035 ...
   1790742.6854019   2444381.02054639  1938616.511376  ]]
[0 0 0 ... 1 1 1]


## Disgust data and Labels

In [ ]:
# E4 data and labels
df_E4 = df_E.loc[df_E[43] == 4]

E4_feat = df_E4.iloc[:, :42].to_numpy()
E4_pdnc_labels = df_E4.iloc[:, 42].to_numpy()
print(E4_feat)
print(E4_pdnc_labels)

[[1735827.97211399 1200707.52985114 2917320.06478765 ... 3170425.50588604
  3496354.78180541 3836369.81104822]
 [1868783.17074136 1554773.53838895 2834645.0818942  ... 3138186.11553765
  4559156.47893681 3639617.74003028]
 [2114898.02299108 1406177.5398297  1884211.26888396 ... 4954730.81597899
  7242831.68930584 4890551.31453686]
 ...
 [1700842.46042163 6352672.12110318 5531594.9159029  ... 2341584.78446318
  4814614.80576291 3163822.69356592]
 [ 592228.59734807 3803730.88368415 2992694.00723206 ... 1886902.30114511
  3488411.51898871 2688179.50316732]
 [1201178.97043846 3184602.35040668 4868910.41587371 ... 2024963.2922791
  3283207.47651031 2613782.81167409]]
[0 0 0 ... 1 1 1]


## Surprise data and Labels

In [ ]:
# E5 data and labels
df_E5 = df_E.loc[df_E[43] == 5]

E5_feat = df_E5.iloc[:, :42].to_numpy()
E5_pdnc_labels = df_E5.iloc[:, 42].to_numpy()
print(E5_feat)
print(E5_pdnc_labels)

[[2705647.63740691 7614561.35229839 2896327.97342155 ... 5337102.51318908
  7365299.70438379 4446497.04382937]
 [2146372.30383579 1722930.79599963 2463109.86055816 ... 3168651.42870529
  3606893.43783328 3695114.62205519]
 [1298708.69965229  926957.78640889 1850467.0810028  ... 2659984.45982001
  4021209.14283063 3551418.08939167]
 ...
 [ 306459.82414947 2883260.54734368 2582440.19823904 ... 1939294.65185026
  3922310.85472405 3141703.68095344]
 [ 573546.50854835 3047755.40292725 3372845.09344594 ... 2312072.21305947
  3613550.72091806 2333218.04525742]
 [1284455.62414179 6281068.60658573 5240057.40779115 ... 2184679.58486214
  3431669.21125823 2367118.53591237]]
[0 0 0 ... 1 1 1]


## Anger data and labels

In [ ]:
# E6 data and labels
df_E6 = df_E.loc[df_E[43] == 6]

E6_feat = df_E6.iloc[:, :42].to_numpy()
E6_pdnc_labels = df_E6.iloc[:, 42].to_numpy()
print(E6_feat)
print(E6_pdnc_labels)

[[1654718.19181436 1624326.37070021 1746570.32306364 ... 2296870.83212576
  3652810.14523496 3684317.02531843]
 [1663357.93300488 1354573.25370729 1145529.82844436 ... 2163848.59786554
  3193015.1818297  2904012.89778867]
 [2589571.08102297  906329.98903301 2771745.07912172 ... 2594340.03807387
  3970258.55577857 3880563.33424702]
 ...
 [3598508.05616058 3498143.20597566 3069446.06854017 ... 2194891.70481264
  4535863.58326616 4047233.55456937]
 [5381991.11344105 4112561.26143319 3849977.9479376  ... 2225377.84184517
  3624477.85673353 3532745.80018185]
 [5335352.63813015 2215708.56991858 2325403.04747604 ... 2870307.90982052
  3915914.3011482  4122432.02235338]]
[0 0 0 ... 1 1 1]


## PD data and Labels

In [ ]:
# PD features and labels

df_E = pd.DataFrame(feature_mat)
df_E[len(df_E.columns)] = df_y_pdnc
df_E[len(df_E.columns)] = df_y_multi
df_E[len(df_E.columns)] = df_y_hvlv
df_E[len(df_E.columns)] = df_y_hala
df_E

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45
0,2.383256e+06,2.384181e+06,1.480505e+06,5.044017e+05,1.614169e+06,5.449679e+06,2.439544e+09,3.551281e+06,2.742891e+06,4.389808e+06,2.854948e+06,2.120282e+06,4.960089e+06,1.389415e+06,9.719958e+06,6.596185e+06,4.929268e+06,1.151921e+06,5.958606e+06,6.303287e+06,1.675759e+09,6.354501e+06,5.911825e+06,1.006109e+07,9.625362e+06,9.136105e+06,1.337641e+07,1.055725e+07,3.638773e+06,2.731453e+06,1.798871e+06,5.902573e+05,3.834957e+06,1.477528e+06,2.219181e+08,2.091681e+06,2.397671e+06,4.816395e+06,3.426161e+06,2.949219e+06,5.552445e+06,4.276479e+06,0,1,0,0
1,1.923588e+06,1.524711e+06,1.440902e+06,3.256426e+05,7.318042e+05,3.087788e+06,1.261696e+09,2.143947e+06,1.504686e+06,2.848900e+06,2.101133e+06,2.499108e+06,3.135291e+06,2.120634e+06,8.197949e+06,7.568225e+06,4.702906e+06,1.204504e+06,5.709211e+06,4.959801e+06,1.217402e+09,5.941888e+06,6.757709e+06,1.110014e+07,7.717924e+06,7.920792e+06,1.288577e+07,9.830794e+06,3.105226e+06,2.053054e+06,1.891156e+06,5.694966e+05,3.100698e+06,2.202368e+06,4.545286e+08,1.926261e+06,2.343108e+06,3.809259e+06,3.366923e+06,2.768477e+06,4.620528e+06,4.353007e+06,0,1,0,0
2,1.668044e+06,1.478329e+06,1.757222e+06,6.122744e+05,9.275701e+05,7.776523e+05,2.684293e+06,2.851677e+06,1.945200e+06,3.285287e+06,3.015931e+06,2.658790e+06,4.325973e+06,2.007657e+06,1.043550e+07,8.950890e+06,5.420378e+06,1.516883e+06,5.453139e+06,2.791502e+06,3.654184e+06,4.423931e+06,4.316393e+06,9.269605e+06,9.245118e+06,9.490669e+06,1.286657e+07,1.149921e+07,2.970087e+06,2.570945e+06,1.600235e+06,8.246155e+05,2.891252e+06,1.227311e+06,1.408648e+06,1.992944e+06,2.630470e+06,4.178466e+06,2.699208e+06,2.248585e+06,4.169795e+06,3.300814e+06,0,1,0,0
3,1.684984e+06,1.299825e+06,1.674538e+06,8.590332e+05,1.447196e+06,7.907034e+05,3.547081e+06,2.602950e+06,1.910653e+06,2.022667e+06,1.801569e+06,2.321919e+06,2.115963e+06,1.578641e+06,7.095673e+06,6.089965e+06,5.110163e+06,1.701532e+06,5.371280e+06,2.675418e+06,5.143468e+06,5.196501e+06,6.448093e+06,1.215845e+07,1.288551e+07,9.064018e+06,2.120405e+07,1.423761e+07,3.202381e+06,2.463451e+06,2.165446e+06,8.934702e+05,3.179321e+06,1.313334e+06,2.098943e+06,2.789685e+06,3.164462e+06,3.807888e+06,3.326751e+06,2.698125e+06,5.405456e+06,4.884444e+06,0,1,0,0
4,1.724961e+06,1.203209e+06,1.368252e+06,6.001277e+05,8.755434e+05,9.193555e+05,3.884886e+06,3.098001e+06,2.062365e+06,2.605652e+06,2.237880e+06,2.249608e+06,2.874567e+06,1.554070e+06,9.520739e+06,6.982346e+06,5.970882e+06,1.545010e+06,5.184717e+06,2.809466e+06,4.850910e+06,4.668346e+06,4.853158e+06,9.000549e+06,7.930759e+06,7.515024e+06,1.058684e+07,8.635522e+06,2.584456e+06,1.980530e+06,1.637318e+06,7.634939e+05,3.983874e+06,1.244363e+06,1.836200e+06,1.726712e+06,2.070195e+06,4.010430e+06,2.271633e+06,2.495437e+06,3.310267e+06,2.922425e+06,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14795,4.525731e+06,6.537415e+06,4.522136e+06,6.006356e+06,6.449785e+06,2.515943e+06,3.328479e+06,8.955641e+06,1.307917e+07,1.687076e+07,1.253696e+07,8.612460e+06,1.449803e+07,9.774463e+06,4.484698e+06,7.807537e+06,4.576137e+06,8.508039e+06,2.949590e+07,7.940605e+06,5.288101e+06,5.691518e+06,7.488903e+06,1.497058e+07,1.380562e+07,5.868346e+06,1.061096e+07,8.779781e+06,2.454194e+06,4.034188e+06,2.457625e+06,7.072872e+06,1.339116e+07,4.845229e+06,2.666139e+06,2.680539e+06,3.089263e+06,6.070497e+06,4.835209e+06,2.558577e+06,3.850525e+06,3.639372e+06,1,6,0,1
14796,2.805448e+06,2.492731e+06,2.754028e+06,2.873462e+06,2.290563e+07,1.939627e+06,2.227679e+06,3.804609e+06,5.123143e+06,8.182871e+06,8.368253e+06,5.403316e+06,6.734352e+06,5.761449e+06,4.731119e+06,7.840018e+06,4.806260e+06,1.310843e+07,5.250813e+07,8.405943e+06,5.301365e+06,6.636248e+06,8.359975e+06,1.618225e+07,1.476195e+07,6.338359e+06

In [ ]:
df_PD = df_E.loc[df_E[42] == 1] #rows with PD entry

pd_feat = df_PD.iloc[:, :42].to_numpy()
pd_multi_labels = df_PD.iloc[:, 43].to_numpy() #multi labels column
pd_hvlv_labels = df_PD.iloc[:, 44].to_numpy() #hvlv labels
pd_hala_labels = df_PD.iloc[:, 45].to_numpy() #hala labels

## HC data and labels

In [ ]:
# NC feat and labels

df_NC = df_E.loc[df_E[42] == 0] #rows with NC entry

nc_feat = df_NC.iloc[:, :42].to_numpy()
nc_multi_labels = df_NC.iloc[:, 43].to_numpy() #multi labels column
nc_hvlv_labels = df_NC.iloc[:, 44].to_numpy() #hvlv labels
nc_hala_labels = df_NC.iloc[:, 45].to_numpy() #hala labels

In [ ]:
# save all these to a .mat file

savemat('/content/drive/MyDrive/EEG Analysis/Features/Spectral/spectral_feat_tensor_full_with_full_labels.mat',
        {'full_feat': full_feat, 'full_pdnc_labels': full_pdnc_labels,
         'full_hvlv_labels' : full_hvlv_labels, 'full_hala_labels':full_hala_labels,
         'full_multi_labels':full_multi_labels, 'E1_feat':E1_feat, 'E1_pdnc_labels':E1_pdnc_labels,
         'E2_feat':E2_feat, 'E2_pdnc_labels':E2_pdnc_labels, 'E3_feat':E3_feat,
         'E3_pdnc_labels':E3_pdnc_labels, 'E4_feat':E4_feat, 'E4_pdnc_labels':E4_pdnc_labels,
         'E5_feat':E5_feat, 'E5_pdnc_labels':E5_pdnc_labels, 'E6_feat':E6_feat,
         'E6_pdnc_labels':E6_pdnc_labels, 'pd_feat':pd_feat, 'pd_hvlv_labels':pd_hvlv_labels,
         'pd_hala_labels':pd_hala_labels, 'pd_multi_labels':pd_multi_labels, 'nc_feat':nc_feat,
         'nc_hvlv_labels':nc_hvlv_labels, 'nc_hala_labels': nc_hala_labels,
         'nc_multi_labels':nc_multi_labels, 'seconds':seconds, 'sampling_rate':sampling_rate, 
         'order':7})

# Archive

In [ ]:
labels_class = {43: {'E1': 1, 'E2':2, 'E3':3, 'E4':4, 'E5':5, 'E6':6},42: {'PD': 1, 'NC':0}}
df_y_pdnc = pd.DataFrame(y_vec_pdnc, dtype='string')
df_y_emot = pd.DataFrame(y_vec_emot, dtype='string')

In [ ]:
#Write to csv
df = pd.DataFrame(feature_mat)
labels_class = {43: {'E1': 1, 'E2':2, 'E3':3, 'E4':4, 'E5':5, 'E6':6},42: {'PD': 0, 'NC':1}}
df_y_pdnc = pd.DataFrame(y_vec_pdnc, dtype='string')
df_y_emot = pd.DataFrame(y_vec_emot, dtype='string')

df[len(df.columns)]=df_y_pdnc #add pdnc labels
df[len(df.columns)]=df_y_emot #add emotions labels
df_new = df.replace(labels_class)
print(df_new.head())

df_new.to_csv('/content/drive/MyDrive/EEG Analysis/Features/spectral_O7_feat_with_labels_full.csv', index=False, header=False)

              0              1              2   ...            41  42  43
0  1.411350e+118  2.247787e+118  8.387240e+117  ...  8.511340e+95   1   1
1  5.295146e+116  1.199026e+111  1.469749e+114  ...  3.416299e+96   1   1
2  9.522396e+117  7.019118e+117  1.453671e+117  ...  1.551327e+94   1   1
3  3.993833e+116  8.286254e+116  2.417270e+117  ...  6.636988e+99   1   1
4  2.299258e+117  2.189563e+115  3.302442e+116  ...  1.047104e+95   1   1

[5 rows x 44 columns]


In [ ]:
#E1 data
dfE1 = df_new.loc[df_new[43] == 1]
print(dfE1.head())
dfE1.to_csv('/content/drive/MyDrive/EEG Analysis/spectral_feat_with_labels_E1.csv', index=False, header=False)

In [ ]:
#E2 data
dfE2 = df_new.loc[df_new[43] == 2]
print(dfE2.head())
dfE2.to_csv('/content/drive/MyDrive/EEG Analysis/spectral_feat_with_labels_E2.csv', index=False, header=False)

In [ ]:
#E3 data
dfE3 = df_new.loc[df_new[43] == 3]
print(dfE3.head())
dfE3.to_csv('/content/drive/MyDrive/EEG Analysis/spectral_feat_with_labels_E3.csv', index=False, header=False)

In [ ]:
#E4 data
dfE4 = df_new.loc[df_new[43] == 4]
print(dfE4.head())
dfE4.to_csv('/content/drive/MyDrive/EEG Analysis/spectral_feat_with_labels_E4.csv', index=False, header=False)

In [ ]:
#E5 data
dfE5 = df_new.loc[df_new[43] == 5]
print(dfE5.head())
dfE5.to_csv('/content/drive/MyDrive/EEG Analysis/spectral_feat_with_labels_E5.csv', index=False, header=False)

In [ ]:
#E6 data
dfE6 = df_new.loc[df_new[43] == 6]
print(dfE6.head())
dfE6.to_csv('/content/drive/MyDrive/EEG Analysis/spectral_feat_with_labels_E6.csv', index=False, header=False)

In [ ]:
def reduce_dim(X, y, pca_thresh, Scaler):
  y = np.ravel(y)
  if Scaler=="MinMax":
    from sklearn.preprocessing import MinMaxScaler
    sc = MinMaxScaler()
    X = sc.fit_transform(X)
  elif Scaler=="MaxAbs":
    from sklearn.preprocessing import MaxAbsScaler
    sc = MaxAbsScaler()
    X = sc.fit_transform(X)
  elif Scaler == "Standard":
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X = sc.fit_transform(X)
  elif Scaler == "Normalizer":
    from sklearn.preprocessing import Normalizer
    sc = Normalizer()
    X = sc.fit_transform(X)
  else:
    pass
  if pca_thresh<1:
    pca = PCA(pca_thresh)
    X = pca.fit_transform(X)
  return X, y

def NB_classifier(X, y, cv, grid_search):
  if grid_search:
    nb_classifier = GaussianNB()
    params_NB = {'var_smoothing': np.logspace(0,-9, num=100)}
    gs_NB = GridSearchCV(estimator=nb_classifier, 
                         param_grid=params_NB, cv=cv,   # use any cross validation technique 
                         verbose=1,
                         n_jobs=2,
                         scoring='accuracy') 
    gs_NB.fit(X, y)
    print("Best parameters:",gs_NB.best_params_)
    var_smoothing = gs_NB.best_params_.get('var_smoothing')
  else:
    var_smoothing = 1e-9
  # your classifier, assume GaussianNB here for non-integer data X
  estimator = GaussianNB(var_smoothing=var_smoothing)
  # generate your cross-validation prediction with 10 fold Stratified sampling
  y_pred = cross_val_predict(estimator, X, y, cv=cv)

  # generate report
  clf_report = classification_report(y, y_pred)
  conf_mat = confusion_matrix(y, y_pred, labels = [1,0])
  return clf_report, conf_mat

def SVM_classifier(X, y, cv, kernel, grid_search):
  if grid_search:
    Cs = [0.001, 0.01, 0.1, 1, 10, 100]
    gammas = [0.001, 0.01, 0.1, 1, 10]
    p_grid = {'C': Cs, 'gamma' : gammas}
    svm_classifier = SVC()
    params_svm = GridSearchCV(estimator=svm_classifier, param_grid=p_grid,
                              cv=cv, refit=True,verbose=1, scoring='accuracy', n_jobs = 8)
    params_svm.fit(X, y)
    print('Best parameters:',params_svm.best_params_)
    C = params_svm.best_params_.get('C')
    gamma = params_svm.best_params_.get('gamma')
    estimator = SVC(kernel=kernel, C=C, gamma = gamma)
    y_pred = cross_val_predict(estimator, X, y, cv=cv)
    # generate report
    clf_report = classification_report(y, y_pred)
    conf_mat = confusion_matrix(y, y_pred, labels = [1,0])
    return clf_report, conf_mat

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/EEG Analysis/spectral_feat_with_labels_full.csv', header = None)
dfx = df.iloc[:, :-2]
dfy = df.iloc[:, -2]
X_full,y_full = reduce_dim(dfx, dfy, 1, 'Normalizer')

In [ ]:
clf_report, conf_mat = SVM_classifier(X_full, y_full, cv=10, kernel='linear', grid_search=1)
print(clf_report)
print(conf_mat)

## Generate Tensor

In [ ]:
# butter band_pass_filter
def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    out = lfilter(b, a, data)
    return out

def theta_BP(data):
    return butter_bandpass_filter(data, 4, 8, 128)

def alpha_BP(data):
    return butter_bandpass_filter(data, 8, 12, 128)

def beta_BP(data):
    return butter_bandpass_filter(data, 12, 30, 128)

#band power AKA sum of squares of fft coeff

def average_intensity(band):
    return sum(np.square(abs(band)))


# theta alpha beta calc
def theta_alpha_beta_calc(data_eeg,max_channel,instance_no):
    thetas = []
    alphas = []
    betas = []

    for i in range(0,max_channel,1):
        data = data_eeg[i,64:,instance_no]
        thetas.append(average_intensity(fft(theta_BP(data))))
        alphas.append(average_intensity(fft(alpha_BP(data))))
        betas.append(average_intensity(fft(beta_BP(data))))

    return np.concatenate((thetas,alphas,betas))


# making feature matrics
def feature_matrix_calc(eeg_data,max_instance):
    feature_matrix = []
    for j in range(0,max_instance,1):
        feature_matrix.append(theta_alpha_beta_calc(eeg_data,14,j))
    return np.array(feature_matrix)


def feature_eeg(dir):

    final_feature_mat = []
    for filename in sorted(os.listdir(dir)):
        if filename.endswith(".mat"): 
            print('Genrating Features for file = ',filename)
            M_data = loadmat(os.path.join(dir, filename))
            eeg_data = M_data[filename[:-4]]
            final_feature_mat.append(feature_matrix_calc(eeg_data,eeg_data.shape[2]))
        else:
            continue
    return np.array(final_feature_mat)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/EEG Analysis/Features/Spectral/spectral_feat_with_labels_full.csv', header = None)

In [ ]:
dfx = df.iloc[:, :-2]
dfpdnc = df.iloc[:, -2]
dfemot = df.iloc[:,-1]

In [ ]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43
0,2.391639e+06,2.368687e+06,1.485694e+06,502253.248689,1.638283e+06,5.681372e+06,2.523674e+09,3.622339e+06,2.793972e+06,4.483388e+06,2.928614e+06,2.162836e+06,5.024249e+06,1.419041e+06,9.920336e+06,6.715533e+06,5.019130e+06,1.195103e+06,6.039012e+06,6.343516e+06,1.695073e+09,6.402127e+06,5.966517e+06,1.015388e+07,9.681612e+06,9.208658e+06,1.352313e+07,1.060274e+07,3.786348e+06,2.793536e+06,1.845843e+06,597154.332540,3.859187e+06,1.506967e+06,2.257751e+08,2.153271e+06,2.431659e+06,4.878220e+06,3.544297e+06,3.062159e+06,5.684306e+06,4.420148e+06,1,1
1,1.974596e+06,1.541213e+06,1.470997e+06,332338.799394,7.789520e+05,3.115665e+06,1.271686e+09,2.194282e+06,1.544779e+06,2.908081e+06,2.140600e+06,2.528012e+06,3.173847e+06,2.162819e+06,8.253733e+06,7.646001e+06,4.723286e+06,1.217730e+06,5.729343e+06,5.015398e+06,1.227609e+09,5.961030e+06,6.782498e+06,1.117885e+07,7.761471e+06,7.977064e+06,1.294051e+07,9.829545e+06,3.244200e+06,2.158656e+06,1.932587e+06,580023.310214,3.116465e+06,2.222712e+06,4.580256e+08,1.976256e+06,2.407015e+06,3.928589e+06,3.453589e+06,2.844832e+06,4.841104e+06,4.511509e+06,1,1
2,1.759532e+06,1.582391e+06,1.839074e+06,637386.561288,9.741805e+05,8.081683e+05,2.737782e+06,2.957077e+06,1.999352e+06,3.448200e+06,3.032720e+06,2.672514e+06,4.393633e+06,2.041410e+06,1.042859e+07,8.952477e+06,5.486849e+06,1.528587e+06,5.457160e+06,2.799590e+06,3.664624e+06,4.456667e+06,4.373241e+06,9.259490e+06,9.193419e+06,9.487754e+06,1.288606e+07,1.148325e+07,3.096099e+06,2.689449e+06,1.650290e+06,834878.549592,2.936585e+06,1.246444e+06,1.442833e+06,2.062359e+06,2.665046e+06,4.228342e+06,2.870247e+06,2.398064e+06,4.349886e+06,3.499056e+06,1,1
3,1.713177e+06,1.311159e+06,1.685930e+06,872218.630396,1.452215e+06,8.214497e+05,3.626868e+06,2.726177e+06,1.992086e+06,2.134005e+06,1.880457e+06,2.360965e+06,2.221239e+06,1.605013e+06,7.271958e+06,6.134505e+06,5.234838e+06,1.729998e+06,5.460796e+06,2.691084e+06,5.195541e+06,5.395579e+06,6.655626e+06,1.231276e+07,1.299095e+07,9.243109e+06,2.113315e+07,1.421990e+07,3.259330e+06,2.574422e+06,2.206535e+06,909919.844441,3.212940e+06,1.336707e+06,2.125509e+06,2.834169e+06,3.219657e+06,3.926801e+06,3.531680e+06,2.820016e+06,5.798400e+06,5.102994e+06,1,1
4,1.742329e+06,1.226001e+06,1.390048e+06,607704.658649,9.168459e+05,9.736248e+05,3.917719e+06,3.150658e+06,2.129808e+06,2.767264e+06,2.424449e+06,2.305350e+06,3.097731e+06,1.608405e+06,9.595013e+06,6.999730e+06,6.000524e+06,1.555925e+06,5.274316e+06,2.815906e+06,4.898065e+06,4.785666e+06,4.936834e+06,9.120500e+06,7.967023e+06,7.592168e+06,1.063404e+07,8.683603e+06,2.736948e+06,2.077191e+06,1.683320e+06,786344.389831,4.010094e+06,1.287941e+06,1.860846e+06,1.793827e+06,2.125577e+06,4.098022e+06,2.345972e+06,2.533637e+06,3.419074e+06,3.034342e+06,1,1
